In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import Ridge

In [87]:
#load and read the training data
DRW_data_path = "/Users/garrettbrown/Desktop/CS189/DRW Competition/DRW_data/train.parquet"
data = pd.read_parquet(DRW_data_path)

#display the beginning of the data, and the shape
print(data.head())
n = data.shape[0]
d = data.shape[1] - 1
print(f"The total data set has {n} data points with {d} features, the last column is labels")

                     bid_qty  ask_qty  buy_qty  sell_qty   volume        X1  \
2023-03-01 00:00:00   15.283    8.425  176.405    44.984  221.389  0.181844   
2023-03-01 00:01:00   38.590    2.336  525.846   321.950  847.796  0.489497   
2023-03-01 00:02:00    0.442   60.250  159.227   136.369  295.596  0.260121   
2023-03-01 00:03:00    4.865   21.016  335.742   124.963  460.705  0.099976   
2023-03-01 00:04:00   27.158    3.451   98.411    44.407  142.818  0.270893   

                           X2        X3        X4        X5  ...      X772  \
2023-03-01 00:00:00 -0.637860  0.006652  0.136870  0.116698  ...  0.333753   
2023-03-01 00:01:00 -0.075619  0.431594  0.522400  0.475255  ...  0.333657   
2023-03-01 00:02:00 -0.444684  0.100695  0.224729  0.203282  ...  0.333667   
2023-03-01 00:03:00 -0.666728 -0.123858  0.019197  0.014459  ...  0.333174   
2023-03-01 00:04:00 -0.325973  0.116336  0.234311  0.214073  ...  0.333171   

                         X773      X774      X775      X

In [89]:
#Take only the data we want, from the preprocessing
data = data.loc[:, ["X21", "X28", "X77", "X181", "X219", "X287", "X465", "X508", "X757", "X758", "label"]]
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [91]:
#indicate how much we want to set aside for validation
valid_size = 50000

#randomize the order and remove the validation
indices = np.arange(n)
np.random.shuffle(indices)
X_total = X.iloc[indices]
y_total = y.iloc[indices]

X_valid = X_total.iloc[:50000]
y_valid = y_total.iloc[:50000]
X = X_total.iloc[50000:]
y = y_total.iloc[50000:]

In [93]:
#create num_bags bagging test sets:
indices = np.arange(n - 50000)
num_bags = 1000
train_sets = []
for i in range(num_bags):
    train = np.random.choice(indices, size = (n - 50000)//4)
    train_sets.append(train)
train_sets = np.array(train_sets)

In [95]:
#load and read the training data
DRW_test_path = "/Users/garrettbrown/Desktop/CS189/DRW Competition/DRW_data/test.parquet"
test_data = pd.read_parquet(DRW_test_path)
test_data

,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,...,X772,X773,X774,X775,X776,X777,X778,X779,X780,label
ID,,,,,,,,,,,,,,,,,,,,,
1,0.317,8.102,13.164,10.272,23.436,-0.341229,0.041851,-0.020094,-0.206221,-0.297124,...,-0.147911,-0.043417,1.521787,1.548965,1.495735,1.166730,0.281056,-0.187831,-0.599553,0
2,2.608,2.111,123.562,40.163,163.725,-1.029564,-1.382505,-1.214935,-1.020241,-0.960397,...,-0.126703,-0.077090,-0.703054,-0.716951,-0.721292,-0.674619,-0.639318,-0.736268,-0.862220,0
3,2.768,10.787,126.137,118.266,244.403,-2.594090,-5.486158,-4.744466,-3.930152,-3.275324,...,-0.147750,-0.030627,-0.703514,-0.717525,-0.731701,-0.750998,-0.789366,-0.850941,-1.033131,0
4,0.948,12.157,16.069,31.723,47.792,0.240745,0.997585,1.028965,1.081052,0.811895,...,-0.136737,-0.033380,1.521167,1.551771,1.582833,1.625830,1.762155,1.911924,1.962445,0
5,1.084,3.493,32.679,37.327,70.006,0.067189,0.772852,0.772152,0.714846,0.514422,...,-0.218991,-0.004915,-0.703161,-0.716900,-0.714699,-0.652209,-0.623165,-0.699887,-0.640094,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538146,1.339,8.653,28.421,15.565,43.986,0.074522,0.025235,-0.218638,-0.253810,-0.136793,...,-0.197908,-0.069053,-0.703431,-0.717405,-0.731374,-0.740635,-0.723005,-0.785533,-0.907044,0
538147,4.377,5.656,18.859,28.044,46.903,-0.893477,-0.173110,-0.199308,-0.296618,-0.414269,...,-0.133929,-0.067185,-0.561957,-0.635284,-0.675021,-0.689030,-0.662521,-0.564414,-0.164031,0
538148,12.188,7.652,53.858,44.792,98.650,0.548106,-0.375903,-0.773056,-0.626512,-0.486983,...,-0.127859,-0.013069,-0.703507,-0.717491,-0.731643,-0.750842,-0.786122,-0.762655,-0.503203,0


In [97]:
test_data = test_data.loc[:, ["X21", "X28", "X77", "X181", "X219", "X287", "X465", "X508", "X757", "X758"]]
test_predictions = np.zeros(test_data.shape[0])

In [99]:
#we're going to take alpha = 0
#train OLS for each test set, average the predictions
for train in train_sets:
    X_bag = X.iloc[train]
    y_bag = y.iloc[train]
    ridge = Ridge(alpha = 0)
    ridge.fit(X_bag, y_bag)
    test_predictions += ridge.predict(test_data)
test_predictions = test_predictions / num_bags

In [101]:
test_predictions = pd.DataFrame(test_predictions)
test_predictions.index = [i for i in range(1, 538151)]
test_predictions.columns = ["prediction"]
test_predictions.index.name = "ID"
test_predictions

,prediction
ID,
1,0.172979
2,0.117335
3,0.027747
4,0.027123
5,-0.077346
...,...
538146,-0.211157
538147,-0.088922
538148,-0.165807


In [103]:
test_predictions.to_csv("submission.csv")